gemaakt voor LIWO door [David Haasnoot](d.haasnoot@hkv.nl)

In [3]:
import dotenv
from pathlib import Path

In [4]:
from update_local_LDO_custom import haal_token_op, export_uit_LDO_custom, get_layer_names_from_scenario


Zie `readme.md` voor uitgebreid stappen plan voor het aanmaken van een api key.

In het kort
- Login & ga naar https://www.overstromingsinformatie.nl/auth/
- Onder `V1`, `POST auth/v1/personalapikeys`, klik op `try it out` en vervang body met:
```json
{
  "scope": "*:readwrite",
  "name": "personalAPIkeyLIWOexport...",
  "expiry_date": "2029-12-31T23:59:59.037Z",
  "revoked": false
}
```
- Pas de datum en name aan waar nodig, klik op `Excecute` in het blauw. In de response body staat: 
```json
{
...
  "key": "xxxxxx.yyyyyyyyyyyyyyy",
...
}
```
- Bewaar die hele `'key'` in een bestand die `.env` heet, zie `example.env` voor het formaat.

In [5]:
# haal de API key op uit de .env file
if dotenv.load_dotenv():
    environmental_variables = dotenv.dotenv_values()
    LDO_api_key = environmental_variables["LDO_api_key"]
# Of zet hier de API key handmatig in
# LDO_api_key = "abcd"
headers = haal_token_op(LDO_api_key)
current_dir = Path.cwd()

In [6]:
import warnings
import requests

server = "https://www.overstromingsinformatie.nl"


def haal_scenarios_op(maximum: int, headers: dict, extra_filter: str = "") -> list:
    """ "Haal de scenario ids op"""
    limit_per_request = 100
    offset = 0
    beschikbare_scenario_ids = get_scenario_list(
        offset, limit_per_request, maximum, headers, extra_filter=extra_filter
    )
    return beschikbare_scenario_ids


def get_scenario_list(offset, limit_per_request, maximum, headers, extra_filter=""):
    mode = "public"
    scenario_ids = []

    for offset in range(0, maximum, limit_per_request):
        scenario_ids += get_scenario_subset(
            mode, limit_per_request, offset, headers, extra_filter=extra_filter
        )

    return scenario_ids


def get_scenario_subset(mode, limit_per_request, offset, headers, extra_filter=""):
    """Get list of scenarios status landelijk gebruik (mode=public)"""
    response = requests.get(
        f"{server}/api/v1/scenarios?mode={mode}&limit={limit_per_request}&offset={offset}&order_by=id{extra_filter}",
        headers=headers,
    )
    if response.status_code == 200:
        result = response.json()
        scenario_list = list(
            (
                item.get("id")
                for item in result["items"]
                if item.get("status") == "quality_checked"
            )
        )
    else:
        warnings.warn(f"{response.status_code}:{response.text}")
        scenario_list = []
    return scenario_list

In [7]:
maximum = 10_000
geo_filter = "&bbox=1.9492904467636265%2C50.770023203519315%2C8.995291621720668%2C52.492957339148916"
extra_filer = geo_filter + "&gebiedsnaam=BRS%20gebied%20Brabantse%20Delta&gebiedsnaam=BRS%20gebied%20Noord-Brabant%20Oost&gebiedsnaam=BRS%20gebied%20Overijsselse%20Vecht&gebiedsnaam=BRS%20gebied%20Rivierenland&gebiedsnaam=BRS%20gebied%20Vallei%20en%20Veluwe"
beschikbare_scenario_ids_geo = haal_scenarios_op(maximum, headers, extra_filter=extra_filer)

In [8]:
len(beschikbare_scenario_ids_geo)

27

In [9]:
maximum = 10_000
extra_filer =  "&gebiedsnaam=BRS%20gebied%20Brabantse%20Delta&gebiedsnaam=BRS%20gebied%20Noord-Brabant%20Oost&gebiedsnaam=BRS%20gebied%20Overijsselse%20Vecht&gebiedsnaam=BRS%20gebied%20Rivierenland&gebiedsnaam=BRS%20gebied%20Vallei%20en%20Veluwe"
beschikbare_scenario_ids = haal_scenarios_op(maximum, headers, extra_filter=extra_filer)

In [10]:
for id in set(beschikbare_scenario_ids).difference(set(beschikbare_scenario_ids_geo)):
    print(f"https://www.overstromingsinformatie.nl/scenarios/{id}")

https://www.overstromingsinformatie.nl/scenarios/111312
https://www.overstromingsinformatie.nl/scenarios/111313
https://www.overstromingsinformatie.nl/scenarios/113715
https://www.overstromingsinformatie.nl/scenarios/113716


Kies welke scenarios gedownload worden:

In [11]:
nieuwe_scenarios = beschikbare_scenario_ids# [-2:]  # de laatste twee scenario's


In [12]:


df_layer_names = get_layer_names_from_scenario( nieuwe_scenarios, headers=headers,)

Exporteert de gewenste bestanden (kan lang duren)

In [ ]:
lst_zips_nieuwe_export = export_uit_LDO_custom(
                                        df_layer_names=df_layer_names,
                                        work_dir=current_dir,
                                        headers=headers,
)

0it [00:00, ?it/s]

23it [27:35, 139.66s/it]